# Sensitivity analysis with uncertainty

This notebook illustrates how to do uncertainty analysis using this library.

The library use metadata to handle uncertinaty information. There are 3 uncertainty strategy defined in this library:
1) `uniformly`: This strategy add the same type/value of the uncertainty to a matrix, but you can add uncertainty for all matricies or only one/two of the matrices.
2) `columnwise`: This strategy add the same type/value of the uncertainty to the same column of a matrix, different column can have different type/value of uncertainty. To use this stragety, your uncertainty input should be defined in the file.
3) `itemwise`: This strategy add different type/value of the uncertainty different element in the matrix. To use this stragety, your uncertainty input should be defined in the file.

Note: 
  - For strategy 2) and 3), only technosphere and biosphere matrices are supported.
  - `itemwise` recommends apply only to the foreground system, considering the amount of data that introduces uncertainty for both systems. The library does not specifically handle this situation.

### Preparation

1. Navigate to the project that contains the ecoinvent database (ecoinvent database is not necessary if you don't need to find characterization factors through Brightway).

In [1]:
import bw2data as bd
import bw2io as bi

bd.projects.set_current("advlca25")

bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb

2. Import required libraries

In [2]:
import os
from bamboo_lca.background_importer import *
from bamboo_lca.foreground_importer import *
from bamboo_lca.datapackage_builder import *
from bamboo_lca.uncertainty_handler import *
from bamboo_lca.lca_wrapper import *
from bamboo_lca.uncertainty_importer import *

3. Define required constants

In [3]:
# This is the biosphere emissions.
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')

# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FOREGROUND_1 = os.path.join(os.getcwd(), "data/foreground_system_1.csv")
FOREGROUND_2 = os.path.join(os.getcwd(), "data/foreground_system_2.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix  
The characterization factors can be extracted from the ecoinvent biosphere database or from `CF_MAPPING_FILE` file.

- If you already have required characterization factors, then you just need to add the characterization factors to the `CFs` column of `CF_MAPPING_FILE` file. Run Option 1 code.
- Otherwise, you need to extract characterization factors from the ecoinvent biosphere database. In this case, you must have the ecoinvent databases imported into Brightway. Run Option 2 code.

Option1: Get characterization factor matrix from the file directly.

In [6]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you can omit some parameters.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00e+00 1.00e+00 1.00e+00 1.00e+00 2.70e+01 2.98e+01 2.98e+01 2.98e+01
 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.73e+02
 2.73e+02 2.52e+04]


Option2: Get characterization factor matrix from brightway.

In [7]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 4.30000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 2.34200000e+02
 2.34200000e+02 2.82148093e+04]


### Step 2: Import the foreground database

4. Check out the foreground data

In [8]:
fg_tech_df = pd.read_table(FOREGROUND_2, sep=',')
fg_tech_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


5. Get activities in foreground system.

In [9]:
fg_activities = get_fg_activities(FOREGROUND_2, ",")
fg_activities

['column_1', 'column_2']

6. Get activities in background system.

In [10]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

7. Get foreground system matrices.  
The foreground system is constructed from four matrices: `fgbg`, `fgfg`, `bgfg`, and `bifg`. These matrices are named to reflect their row and column positions. Specifically:

- `fgbg`: This is the matrix locate the foreground row first, then the background column. It indicates the amount of exchange that takes from background to foreground. Normally, there is no such exchange. So, by default this matrix is all zero.
- `fgfg`: This is the matrix locate the foreground row first, then the foreground column. It indicates the amount of exchange that takes from foreground to foreground.
- `bgfg`: This is the matrix locate the background row first, then the foreground column. It indicates the amount of exchange that takes from foreground to background.
- `bifg`: This is the matrix locate the biosphere emission row first, then the foreground column. It indicates the amount of exchange that takes from foreground to biosphere.

In [11]:
fg_importer = ForegroundImporter()

fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_tech_df, GHG, fg_activities, bg_activities)

8. Concatenate foreground system with background system

In [12]:
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change from {tech_matrix.shape} to {full_tech_matrix.shape}.")
print(f"Biosphere shape change from {bio_matrix.shape} to {full_bio_matrix.shape}.")

Technosphere shape change from (76, 76) to (78, 78).
Biosphere shape change from (18, 76) to (18, 78).


### Step 3: Add uncertainty into metadata

9. Get activities for both foreground system and background system.

In [13]:
activities = fg_activities + bg_activities  # note: we always want foreground activities to be in front.
activities[:5]  # only print the first 5 activities

['column_1',
 'column_2',
 'EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying']

10. Store uncertainty information in library metadata

In [14]:
uncertainty_importer = UncertaintyImporter(FOREGROUND_2, ",")
uncertainty_importer.update_metadata_uncertainty(activities, "itemwise")  # here the uncertainty strategy is set to "itemwise" as an example.
uncertainty_importer.metadata

{0: {'Activity name': 'column_1',
  'Activity uncertainty type': 2,
  'Exchange uncertainty amount': [1.111,
   0.0,
   0.0,
   0.0,
   0.0,
   3.33000452,
   1.54,
   1.54,
   0.0,
   1.54,
   1.54,
   4.09770687,
   0.0,
   1.59,
   0.0,
   0.0,
   2.014014413,
   0.0,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   2.674794737,
   0.0,
   2.674794737,
   2.674794737,
   0.0,
   2.674794737,
   2.288684673,
   0.0,
   0.0,
   0.0,
   2.238117654,
   2.699910682,
   0.0,
   1.56,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  'Exchange negative': [False,
   False,
   True,
   True,
   True,
   False,
   True,
   True,
   True,
   False,
   True,
   False,


### Step 4: Build datapackage

11. Prepare datapackage matrix data

In [15]:
dp_builder = DatapackageBuilder()
(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (cf_data, cf_indices) = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, cf_matrix)

12. Build uncertainty arrays for datapackage

In [16]:
uncertainty_handler = UncertaintyHandler()
full_tech_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_tech_data, full_tech_indices, full_tech_flip, "itemwise", fg_num=2, fg_strategy="itemwise")
full_bio_uncertainty_array = uncertainty_handler.add_nonuniform_uncertainty(full_bio_data, full_bio_indices, None, "itemwise", fg_num=2, fg_strategy="itemwise")

13. Check the shape of the uncertainty array.  
- As long as the data shape is identical to the uncertainty shape, it's correct.  
- Note: the shape is not identical to the matrix shape (76 * 76), (20 * 76) or (20), because a sparse matrix does not store zero data.

In [17]:
print(f"The length of technosphere and biosphere array: {full_tech_data.shape, full_bio_data.shape}")
print(f"The length of technosphere and biosphere uncertainty array: {full_tech_uncertainty_array.shape, full_bio_uncertainty_array.shape}")

The length of technosphere and biosphere array: ((5556,), (1291,))
The length of technosphere and biosphere uncertainty array: ((5556,), (1291,))


14. Check the uncertainty data

In [18]:
print(f"Information store in uncertainty array: \n {[dtype[0]for dtype in bwp.UNCERTAINTY_DTYPE]} \n")
print(f"Examples of uncertainty data required in datapackage: \n {full_tech_uncertainty_array[:10]} \n")

Information store in uncertainty array: 
 ['uncertainty_type', 'loc', 'scale', 'shape', 'minimum', 'maximum', 'negative'] 

Examples of uncertainty data required in datapackage: 
 [(2, 0.        , 0.10526051, nan, nan, nan, False)
 (2, 0.        , 0.10526051, nan, nan, nan, False)
 (0, 0.7363383 ,        nan, nan, nan, nan, False)
 (0, 0.00132513,        nan, nan, nan, nan, False)
 (0, 0.00250369,        nan, nan, nan, nan, False)
 (0, 0.00457712,        nan, nan, nan, nan, False)
 (0, 0.00210603,        nan, nan, nan, nan, False)
 (0, 0.00186496,        nan, nan, nan, nan, False)
 (0, 0.00194427,        nan, nan, nan, nan, False)
 (0, 0.00127726,        nan, nan, nan, nan, False)] 



15. Build datapackage

In [19]:
datapackage_data = [(full_tech_data, full_tech_indices, full_tech_flip), (full_bio_data, full_bio_indices), (cf_data, cf_indices)]
uncertainty = [full_tech_uncertainty_array, full_bio_uncertainty_array, None]

dp = dp_builder.prepare_datapackage(datapackage_data, uncertainty)
dp.data

[array([( 0,  0), ( 1,  1), ( 2,  2), ..., (77, 75), (77, 76), (77, 77)],
       dtype=[('row', '<i8'), ('col', '<i8')]),
 array([1.00000000e+00, 1.00000000e+00, 7.36338343e-01, ...,
        6.24171469e-03, 9.40994608e-04, 9.95178298e-01]),
 array([(2, 0.0000000e+00, 0.10526051, nan, nan, nan, False),
        (2, 0.0000000e+00, 0.10526051, nan, nan, nan, False),
        (0, 7.3633832e-01,        nan, nan, nan, nan, False), ...,
        (0, 6.2417146e-03,        nan, nan, nan, nan, False),
        (0, 9.4099459e-04,        nan, nan, nan, nan, False),
        (0, 9.9517828e-01,        nan, nan, nan, nan, False)],
       dtype=[('uncertainty_type', 'u1'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')]),
 array([False, False, False, ...,  True,  True, False]),
 array([(78,  2), (78,  3), (78,  4), ..., (95, 75), (95, 76), (95, 77)],
       dtype=[('row', '<i8'), ('col', '<i8')]),
 array([6.92719535e+04, 2.24902521e+05, 1.58717

### Step 5: Run LCA

16. Choose functional unit

In [20]:
# This is the list includes all activities.
activities = fg_activities + bg_activities 

# We want to set "column_1" as functional unit.
selected_activity = "column_1"

# We need to find the index of the activity in activities.
index = activities.index(selected_activity)

print(f"The index of the 'column_1' is {index}")

# Define the index of the functional unit
functional_unit = {index: 1}
print(f"The functional unit is {functional_unit}")

The index of the 'column_1' is 0
The functional unit is {0: 1}


17. Run static LCA

In [21]:
lca = bc.LCA(
            demand=functional_unit,
            data_objs=[dp],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[index]}")

Brightway calculated lca score: (-78.81960328012741, 'column_1')


18. Run LCA 10 times to see the uncertainty

In [22]:
for i in range(10):
    lca = bc.LCA(
                demand=functional_unit,
                data_objs=[dp],
                use_distributions=True,
            )
    lca.lci()
    lca.lcia()

    print(f"Brightway calculated lca score: {lca.score, activities[index]}")

Brightway calculated lca score: (-68.90978348216971, 'column_1')
Brightway calculated lca score: (-153.94447894853582, 'column_1')
Brightway calculated lca score: (-74.48719330802712, 'column_1')
Brightway calculated lca score: (-80.51324902195626, 'column_1')
Brightway calculated lca score: (285.883182018309, 'column_1')
Brightway calculated lca score: (-108.16099489422879, 'column_1')
Brightway calculated lca score: (-103.24785480664286, 'column_1')
Brightway calculated lca score: (-45.75067151174946, 'column_1')
Brightway calculated lca score: (-87.33751320798945, 'column_1')
Brightway calculated lca score: (-57.586852561008065, 'column_1')


### Step 6: Compare to manually LCA

19. Compare with lca manually.

In [23]:
lca_wrapper = LCAWrapper()

full_tech_matrix_manual = full_tech_matrix.copy()

np.fill_diagonal(full_tech_matrix_manual, -full_tech_matrix_manual.diagonal())
full_tech_matrix_manual = -full_tech_matrix_manual

manual_lca = lca_wrapper.manual_lca(full_tech_matrix_manual, full_bio_matrix, cf_matrix, index)

print(f"Manually calculated lca score: {bg_activities[index]}: {manual_lca}")

Manually calculated lca score: EU28-Agriculture-Forestry-Fishing: -78.81960328012745
